## Prepare data

In [27]:
import numpy as np
import pandas as pd
import pickle, re
from keras import Model
from sklearn.neighbors import KDTree
from processors import QueProc, ProProc
from models import DistanceModel as ContentModel

In [29]:
with open('proc_data/test_que_data.pkl', 'rb') as f:
    train_que_data = pickle.load(f)
with open('proc_data/test_stu_data.pkl', 'rb') as f:
    train_stu_data = pickle.load(f)
with open('proc_data/test_pro_data.pkl', 'rb') as f:
    train_pro_data = pickle.load(f)

In [30]:
que_cols = list(train_stu_data.columns[2:]) + list(train_que_data.columns[2:]) + ['questions_current_time']
pro_cols = list(train_pro_data.columns[2:]) + ['professionals_current_time']
print(len(que_cols), len(pro_cols))

que_content_mask = np.zeros(len(que_cols)-1, dtype=bool) # Change
for i, col in enumerate(que_cols[:-1]): # Change
    if re.search(r'emb', col):
        que_content_mask[i] = True

pro_content_mask = np.zeros(len(pro_cols)-1, dtype=bool) # Change
for i, col in enumerate(pro_cols[:-1]): # Change
    if re.search(r'emb', col):
        pro_content_mask[i] = True
print(que_content_mask.size, pro_content_mask.size)

28 33
27 32


In [47]:
from models_tmp import ContentModel

In [49]:
content_model = ContentModel(
    len(que_cols), que_content_mask,
    len(pro_cols), pro_content_mask,
    10, 5,
)
content_model.load_weights('content_model.h5')

In [58]:
with open('proc_data/test_stu_data.pkl', 'rb') as f:
    stu_data = pickle.load(f)

In [59]:
stu_data

,students_id,students_time,students_location,students_state,students_questions_asked,students_average_question_age,students_average_question_body_length,students_average_answer_body_length,students_average_answer_amount,students_date_joined_time,students_date_joined_doy_sin,students_date_joined_doy_cos,students_previous_question_date_time,students_previous_question_date_doy_sin,students_previous_question_date_doy_cos
0,12a89e96755a4dba83ff03e03043d9c0,NaT,101,39,-0.354836,0.041078,-0.039474,0.004299,0.017147,-3.786064,-0.642175,1.567627,-3.865827,-0.649283,1.602996
1,12a89e96755a4dba83ff03e03043d9c0,2011-12-16 14:26:31,101,39,-0.196354,4.799489,0.392767,-0.698948,-0.113240,-3.786064,-0.642175,1.567627,-3.865827,-0.649283,1.602996
2,efab489b939a45f9a9c2bdda933d8dda,NaT,101,39,-0.354836,0.041078,-0.039474,0.004299,0.017147,-3.746910,-0.180733,1.613058,-3.826097,-0.185226,1.648448
3,efab489b939a45f9a9c2bdda933d8dda,2012-01-06 18:26:32,101,39,-0.196354,4.604485,-0.368596,1.673006,1.304798,-3.746910,-0.180733,1.613058,-3.820675,-0.136124,1.644078
4,b712ae1a97914821ba6294210043a7f2,NaT,100,3,-0.354836,0.041078,-0.039474,0.004299,0.017147,-3.746910,-0.180733,1.613058,-3.826097,-0.185226,1.648448
5,b712ae1a97914821ba6294210043a7f2,2012-01-06 19:49:24,100,3,-0.196354,-0.844513,-1.045364,-0.257036,-0.822259,-3.746910,-0.180733,1.613058,-3.820586,-0.136124,1.644078
6,b712ae1a97914821ba6294210043a7f2,2012-09-07 19:01:57,100,3,-0.037871,-0.852702,-0.866411,-0.257463,-0.822259,-3.746910,-0.180733,1.613058,-3.297963,-1.606926,-0.388187
7,d78a186ccc4d473384ebf02fda4a36c2,NaT,69,1,-0.354836,0.041078,-0.039474,0.004299,0.017147,-3.746910,-0.180733,1.613058,-3.826097,-0.185226,1.648448
8,d78a186ccc4d473384ebf02fda4a36c2,2012-01-06 19:51:21,69,1,-0.196354,-0.837685,-1.299152,-0.077102,0.595779,-3.746910,-0.180733,1.613058,-3.820586,-0.136124,1.644078
9,d78a186ccc4d473384ebf02fda4a36c2,2012-01-06 19:58:28,69,1,-0.037871,-0.841305,-0.924978,-0.099543,0.542268,-3.746910,-0.180733,1.613058,-3.820586,-0.136124,1.644078


---

## Predictor class

In [53]:
class Predictor:
    """
    Class that creates KNN tree for professionals
    and which is used to find closest professionals for a particular question
    """
    
    def __init__(self, content_model: ContentModel):
        """
        Prepare required datasets and create KNN tree for professionals
        based on latent vectors from content model
        :param content_model: compiled model of class ContentModel
        """
        self.content_model = content_model
        
        # load raw datasets
        self.que = pd.read_csv('../../data/questions.csv')
        self.ans = pd.read_csv('../../data/answers.csv')
        self.stu = pd.read_csv('../../data/students.csv')
        self.pro = pd.read_csv('../../data/professionals.csv')
        
        # process date columns
        for df, col in [(self.que, 'questions_date_added'), (self.ans, 'answers_date_added')]:
            df[col] = pd.to_datetime(df[col])
        
        # form question-student pairs dataframe
        que_stu = self.que.merge(self.stu, left_on='questions_author_id', right_on='students_id') \
            [['questions_id', 'students_id']]
        
        # load and merge raw tag datasets
        tags = pd.read_csv('../../data/tags.csv')
        tag_que = pd.read_csv('../../data/tag_questions.csv').merge(
            tags, left_on='tag_questions_tag_id', right_on='tags_tag_id')
        tag_pro = pd.read_csv('../../data/tag_users.csv').merge(
            tags, left_on='tag_users_tag_id', right_on='tags_tag_id')
        
        # append tags to each question
        que_tags = tag_que.groupby('tag_questions_question_id', as_index=False)[['tags_tag_name']] \
            .aggregate(lambda x: ' '.join(set(x))) \
            .rename({'tag_questions_question_id': 'questions_id',
                     'tags_tag_name': 'questions_tags'}, axis=1)
        self.que = self.que.merge(que_tags, how='left', on='questions_id')
        
        # append subscribed tags to each professional
        pro_tags = tag_pro.groupby('tag_users_user_id', as_index=False)[['tags_tag_name']] \
            .aggregate(lambda x: ' '.join(set(x))) \
            .rename({'tag_users_user_id': 'professionals_id',
                     'tags_tag_name': 'professionals_subscribed_tags'}, axis=1)
        self.pro = self.pro.merge(pro_tags, how='left', on='professionals_id')
        
        # load datasets with preprocessed features
        with open('proc_data/test_que_data.pkl', 'rb') as f:
            self.que_data = pickle.load(f)
        with open('proc_data/test_stu_data.pkl', 'rb') as f:
            self.stu_data = pickle.load(f)
        with open('proc_data/test_pro_data.pkl', 'rb') as f:
            self.pro_data = pickle.load(f)
        
        # prepare student features
        stu_dict = {stu: group.iloc[-1, 2:] for stu, group in self.stu_data.groupby('students_id')}
        self.stu_data = pd.DataFrame.from_dict(stu_dict).transpose()
        
        # prepare student features for questions from que_data
        stus = self.que_data.merge(que_stu, how='left', on='questions_id')[['students_id']]
        stu_feat = stus.merge(self.stu_data, how='left', left_on='students_id', right_index=True).values[:, 1:]
        
        # prepare question features and add them to student features
        self.ques = self.que_data['questions_id'].values
        que_feat = self.que_data.values[:, 2:]
        que_feat = np.hstack([stu_feat, que_feat, np.zeros((que_feat.shape[0], 1))])
        
        # prepare professional features
        pro_dict = {pro: group.values[-1, 2:] for pro, group in self.pro_data.groupby('professionals_id')}
        self.pros = np.array(list(pro_dict.keys()))
        pro_feat = np.vstack(pro_dict.values())
        pro_feat = np.hstack([pro_feat, np.zeros((pro_feat.shape[0], 1))])
        
        # create two models that receieve question and professional features respectively
        # and return their latent vectors
        self.que_lat_model = Model(inputs=content_model.inputs[0], outputs=content_model.que_encoded)
        self.pro_lat_model = Model(inputs=content_model.inputs[1], outputs=content_model.pro_encoded)
        
        # compute latent vectors for questions and professionals
        que_lat_vecs = self.que_lat_model.predict(que_feat)
        pro_lat_vecs = self.pro_lat_model.predict(pro_feat)
        
        # create two KNN trees consisting of question and professional latent vectors
        self.que_tree = KDTree(que_lat_vecs)
        self.pro_tree = KDTree(pro_lat_vecs)
        
        # initialize QueProc and ProProc
        self.que_proc = QueProc(oblige_fit=False, path='dump/')
        self.pro_proc = ProProc(oblige_fit=False, path='dump/')
    
    
    def find_pros_by_que(self, que_df: pd.DataFrame, que_tags: pd.DataFrame,
                         top: int=10, expand: bool=False) -> pd.DataFrame:
        """
        Returns top professionals for given questions
        :param que_df: DataFrame of question data
        :param que_tags: DataFrame of question tags
        :param top: how many top professionals to return
        :param expand: whether to add professional data to returned DataFrame
        """
        que_df['questions_date_added'] = pd.to_datetime(que_df['questions_date_added'])
        
        # prepare student features
        stu_feat = que_df[['questions_author_id']] \
            .merge(self.stu_data, how='left', left_on='questions_author_id', right_index=True).values[:, 1:]
        
        print(stu_feat)
        
        # prepare question features and add them to student features
        que_feat = self.que_proc.transform(que_df, que_tags).values[:, 2:]
        
        print(que_feat)
        
        que_feat = np.hstack([stu_feat, que_feat, np.zeros((que_feat.shape[0], 1))])
        
        # get top professionals for questions
        que_lat_vecs = self.que_lat_model.predict(que_feat)
        dists, pros = self.pro_tree.query(que_lat_vecs, k=top)
        pros = self.pros[pros]
        scores = np.exp(-dists)
        ques = que_df['questions_id'].values
        
        # create question-professional-score tuples
        tuples = []
        for i, que in enumerate(ques):
            for j, pro in enumerate(pros[i]):
                tuples.append((que, pro, scores[i, j]))
        
        # create DataFrame from tuples
        score_df = pd.DataFrame(tuples, columns=['questions_id', 'professionals_id', 'professionals_score'])
        
        if expand:
            # add professionals features
            score_df = score_df.merge(self.pro, how='left', on='professionals_id')
        
        return score_df
    
    
    def find_ques_by_que(self, que_df: pd.DataFrame, que_tags: pd.DataFrame,
                         top: int=10, expand: bool=False) -> pd.DataFrame:
        """
        Returns top similar questions for given questions
        :param que_df: DataFrame of question data
        :param que_tags: DataFrame of question tags
        :param top: how many top professionals to return
        :param expand: whether to add professional data to returned DataFrame
        """
        que_df['questions_date_added'] = pd.to_datetime(que_df['questions_date_added'])
        
        # prepare student features
        stu_feat = que_df[['questions_author_id']] \
            .merge(self.stu_data, how='left', left_on='questions_author_id', right_index=True).values[:, 1:]
        
        # prepare question features and add them to student features
        que_feat = self.que_proc.transform(que_df, que_tags).values[:, 2:]
        que_feat = np.hstack([stu_feat, que_feat, np.zeros((que_feat.shape[0], 1))])
        
        # get top similar questions for initial questions
        que_lat_vecs = self.que_lat_model.predict(que_feat)
        dists, sim_ques = self.que_tree.query(que_lat_vecs, k=top)
        sim_ques = self.ques[sim_ques]
        scores = np.exp(-dists)
        ques = que_df['questions_id'].values
        
        # create question-similar_question-score tuples
        tuples = []
        for i, que in enumerate(ques):
            for j, sim_que in enumerate(sim_ques[i]):
                tuples.append((que, sim_que, scores[i, j]))
        
        # create DataFrame from tuples
        score_df = pd.DataFrame(tuples, columns=['initial_questions_id', 'questions_id', 'questions_score'])
        
        if expand:
            # add similar question features
            score_df = score_df.merge(self.que, how='left', on='questions_id')
        
        return score_df
    
    
    def convert_que_dict(self, que_dict: dict) -> (pd.DataFrame, pd.DataFrame):
        """
        Converts dictionary of question data into desired form
        :param que_dict: dictionary of question data
        """
        # get DataFrame from dict
        que_df = pd.DataFrame.from_dict(que_dict)
        ques = que_df['questions_id'].values
        
        # create question-tag tuples
        tuples = []
        for i, tags in enumerate(que_df['questions_tags'].values):
            que = ques[i]
            for tag in tags.split(' '):
                tuples.append((que, tag))
        
        # create DataFrame from tuples
        que_tags = pd.DataFrame(tuples, columns=['tag_questions_question_id', 'tags_tag_name'])
        que_df.drop(columns='questions_tags', inplace=True)
        
        return que_df, que_tags
    
    
    def find_ques_by_pro(self, pro_df: pd.DataFrame, pro_tags: pd.DataFrame,
                         top: int=10, expand: bool=False) -> pd.DataFrame:
        """
        Returns top questions for given professionals
        :param pro_df: DataFrame of professional data
        :param pro_tags: DataFrame of professional subscribed tags
        :param top: how many top professionals to return
        :param expand: whether to add professional data to returned DataFrame
        """
        pro_df['professionals_date_joined'] = pd.to_datetime(pro_df['professionals_date_joined'])
        
        # prepare professional features
        pro_feat = self.pro_proc.transform(pro_df, self.que, self.ans, pro_tags).values[:, 2:]
        pro_feat = np.hstack([pro_feat, np.zeros((pro_feat.shape[0], 1))])
        
        # get top questions for professionals
        pro_lat_vecs = self.pro_lat_model.predict(pro_feat)
        dists, ques = self.que_tree.query(pro_lat_vecs, k=top)
        ques = self.ques[ques]
        scores = np.exp(-dists)
        pros = pro_df['professionals_id'].values
        
        # create professional-question-score tuples
        tuples = []
        for i, pro in enumerate(pros):
            for j, que in enumerate(ques[i]):
                tuples.append((pro, que, scores[i, j]))
        
        # create DataFrame from tuples
        score_df = pd.DataFrame(tuples, columns=['professionals_id', 'questions_id', 'questions_score'])
        
        if expand:
            # add question features
            score_df = score_df.merge(self.que, how='left', on='questions_id')
        
        return score_df
    
    
    def convert_pro_dict(self, pro_dict: dict) -> (pd.DataFrame, pd.DataFrame):
        """
        Converts dictionary of professional data into desired form
        :param pro_dict: dictionary of professional data
        """
        # get DataFrame from dict
        pro_df = pd.DataFrame.from_dict(pro_dict)
        pros = pro_df['professionals_id'].values
        
        # create professional-tag tuples
        tuples = []
        for i, tags in enumerate(pro_df['professionals_subscribed_tags'].values):
            pro = pros[i]
            for tag in tags.split(' '):
                tuples.append((pro, tag))
        
        # create DataFrame from tuples
        pro_tags = pd.DataFrame(tuples, columns=['tag_users_user_id', 'tags_tag_name'])
        pro_df.drop(columns='professionals_subscribed_tags', inplace=True)
        
        return pro_df, pro_tags

---

## Test Predictor class

In [1]:
from models import DistanceModel

Using TensorFlow backend.


In [4]:
model = DistanceModel(que_dim= 24 - 2 + 8 - 2,
                                  que_input_embs=[102, 42], que_output_embs=[2, 2],
                                  pro_dim=27 - 2,
                                  pro_input_embs=[102, 102, 42], pro_output_embs=[2, 2, 2],
                                  inter_dim=16, output_dim=10)
model.load_weights('dump/model.h5')

In [26]:
import importlib

In [5]:
import predictor

C:\Users\nikitosoleil\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [49]:
importlib.reload(predictor)

<module 'predictor' from 'C:\\Users\\nikitosoleil\\DataRoot\\cv\\cv\\proj\\predictor.py'>

In [50]:
pred = predictor.Predictor(model, 'dump/')

In [51]:
que_dict = {
    'questions_id': ['332a511f1569444485cf7a7a556a5e54'],
    'questions_author_id': ['8f6f374ffd834d258ab69d376dd998f5'],
    'questions_date_added': ['2016-04-26 11:14:26'],
    'questions_title': ['Teacher   career   question'],
    'questions_body': ['What  is  a  maths  teacher?   what  is  a  maths  teacher  useful? #college #professor #lecture'],
    'questions_tags': ['college professor lecture']
}

In [52]:
que_df, que_tags = pred.convert_que_dict(que_dict)

### Recommend the question to professionals

In [53]:
pred.find_pros_by_que(que_df, que_tags)

[[14 10 -0.22259948167676602 -0.5578865162861019 -0.4425965550060855
  -0.06277095584967236]] [[-0.6340867716159001 -0.3775441067623759 -0.3598273694515228
  -0.6401242613792419 -0.1037975326180458 -0.7375578284263611
  -0.7670846581459045 -0.730384886264801 0.3594370186328888
  0.5902867317199707 0.20080788433551788 0.8254199028015137
  0.02776036225259304 0.08866214007139206 -0.36753249168395996
  0.3127608299255371 0.9660110473632812 0.14297381043434143
  -0.15308810770511627 -0.28848111629486084 -0.5757001638412476
  0.020150894299149513]]
[[-0.1515251   0.10362332 -0.00560359  0.3837396   0.22641943 -0.01112039
   0.00892591  0.25103888  0.01351352  0.25848457]]


,questions_id,professionals_id,professionals_score
0,332a511f1569444485cf7a7a556a5e54,[fe5fc534894e45c191e87f8c2ea0c6bd],0.837531
1,332a511f1569444485cf7a7a556a5e54,[22bbfbb66df1454a8eeca5e47900db84],0.818233
2,332a511f1569444485cf7a7a556a5e54,[5f842cba3c844104a9ae9686e425017d],0.802633
3,332a511f1569444485cf7a7a556a5e54,[ccc5108273214a7492b64e1ee480568c],0.794926
4,332a511f1569444485cf7a7a556a5e54,[1cb4dc6d104546c382c58a6dfb3abce1],0.789423
5,332a511f1569444485cf7a7a556a5e54,[fe40a0bcb4ad4f819d308bb1c91ec221],0.789212
6,332a511f1569444485cf7a7a556a5e54,[708dac91e794411880262fd6769828a8],0.781483
7,332a511f1569444485cf7a7a556a5e54,[a6b0200d1b0e4357997028b558db0ab5],0.778286
8,332a511f1569444485cf7a7a556a5e54,[97ac539abf554ca399f1075b0818dc75],0.778146
9,332a511f1569444485cf7a7a556a5e54,[51a38e3c7e264f27a71ccd481ee46321],0.778143


### Find similar questions

In [54]:
pred.find_ques_by_que(que_df, que_tags)

[[14 10 -0.22259948167676602 -0.5578865162861019 -0.4425965550060855
  -0.06277095584967236]] [[-0.6340867716159001 -0.3775441067623759 -0.3598273694515228
  -0.6401242613792419 -0.1037975326180458 -0.7375578284263611
  -0.7670846581459045 -0.730384886264801 0.3594370186328888
  0.5902867317199707 0.20080788433551788 0.8254199028015137
  0.02776036225259304 0.08866214007139206 -0.36753249168395996
  0.3127608299255371 0.9660110473632812 0.14297381043434143
  -0.15308810770511627 -0.28848111629486084 -0.5757001638412476
  0.020150894299149513]]
[[-0.1515251   0.10362332 -0.00560359  0.3837396   0.22641943 -0.01112039
   0.00892591  0.25103888  0.01351352  0.25848457]]


,initial_questions_id,questions_id,questions_score
0,332a511f1569444485cf7a7a556a5e54,[1f06a0e1a7aa45c2a4042772fbf67426],0.543139
1,332a511f1569444485cf7a7a556a5e54,[09b0f89f79c346e6acb69518bb59b4be],0.535604
2,332a511f1569444485cf7a7a556a5e54,[d9b9afca4e8b4483959bffe0d41f8db4],0.535577
3,332a511f1569444485cf7a7a556a5e54,[5d21991825584bfab047ac19add82c81],0.534572
4,332a511f1569444485cf7a7a556a5e54,[f13293a4dc6e49f8a597108188f3e3e3],0.533891
5,332a511f1569444485cf7a7a556a5e54,[9eefc1f7998b4fb18aa8c3c0ba4d2be2],0.533614
6,332a511f1569444485cf7a7a556a5e54,[f3b6cdb747ac49f7b0a35e648630ac46],0.532277
7,332a511f1569444485cf7a7a556a5e54,[f92377f8b5f746df9795526054832ff6],0.532098
8,332a511f1569444485cf7a7a556a5e54,[bc7681202f8b4eebaf55ca49d79042e0],0.532012
9,332a511f1569444485cf7a7a556a5e54,[38cd0a6c4f6c41028692df1aaac49e22],0.530504


In [14]:
%%timeit
pred.find_ques_by_que(que_df, que_tags)

[[14 10 -0.22259948167676602 -0.5578865162861019 -0.4425965550060855
  -0.06277095584967236]] [[-0.6340867716159001 -0.3775441067623759 -0.3598273694515228
  -0.6401242613792419 -0.1037975326180458 -0.7375578284263611
  -0.7670846581459045 -0.730384886264801 0.3594370186328888
  0.5902867317199707 0.20080788433551788 0.8254199028015137
  0.02776036225259304 0.08866214007139206 -0.36753249168395996
  0.3127608299255371 0.9660110473632812 0.14297381043434143
  -0.15308810770511627 -0.28848111629486084 -0.5757001638412476
  0.020150894299149513]]
[[14 10 -0.22259948167676602 -0.5578865162861019 -0.4425965550060855
  -0.06277095584967236]] [[-0.6340867716159001 -0.3775441067623759 -0.3598273694515228
  -0.6401242613792419 -0.1037975326180458 -0.7375578284263611
  -0.7670846581459045 -0.730384886264801 0.3594370186328888
  0.5902867317199707 0.20080788433551788 0.8254199028015137
  0.02776036225259304 0.08866214007139206 -0.36753249168395996
  0.3127608299255371 0.9660110473632812 0.1429738

In [12]:
%load_ext line_profiler

In [13]:
%lprun -f predictor.find_ques_by_que predictor.find_ques_by_que(que_df, que_tags, expand=True)

---

In [15]:
pro_dict = {
    'professionals_id': ['44b2484ecd3642c6a47514f3876cf14a'],
    'professionals_location': ['Cambridge, Massachusetts'],
    'professionals_industry': ['Healthcare, Pharmaceuticals, Life Science'],
    'professionals_headline': ['Scientist and Healthcare Entrepreneur'],
    'professionals_date_joined': ['2012-01-25 20:40:43'],
    'professionals_subscribed_tags': ['pharmaceutical-industry medicine healthcare-it biology science healthcare research']
}

In [17]:
pro_df, pro_tags = predictor.Predictor.convert_pro_dict(pro_dict)

### Recommend questions to the professional

In [21]:
import pandas as pd

In [23]:
ans_df = pd.read_csv('../../data/answers.csv')
que_df = pd.read_csv('../../data/questions.csv')

In [24]:
pred.find_ques_by_pro(pro_df, que_df, ans_df, pro_tags)

,professionals_id,questions_id,questions_score
0,44b2484ecd3642c6a47514f3876cf14a,[13d3fd8c3b0845e1bc3c22633c5feeec],0.370836
1,44b2484ecd3642c6a47514f3876cf14a,[132cca46c7f24c2eac41515741d7f1d4],0.370542
2,44b2484ecd3642c6a47514f3876cf14a,[18e70d75535a43739ee252e577f34f4a],0.370197
3,44b2484ecd3642c6a47514f3876cf14a,[69dd54b0d9ef4cafa3283b0daf3d513f],0.370071
4,44b2484ecd3642c6a47514f3876cf14a,[ea5483f7384645179e00637709959885],0.370069
5,44b2484ecd3642c6a47514f3876cf14a,[ee4024de48b34c3b907ba1d221236f55],0.370027
6,44b2484ecd3642c6a47514f3876cf14a,[78d3a3708ec44c1d89a2a3c21ea35178],0.369878
7,44b2484ecd3642c6a47514f3876cf14a,[3b3951e98b6c4364b5825929169b2216],0.369856
8,44b2484ecd3642c6a47514f3876cf14a,[fa968ccb3d2d44bfa6750b43e325b902],0.369824
9,44b2484ecd3642c6a47514f3876cf14a,[541d90ef44704428a6f433e443db8c62],0.369785


In [18]:
%%timeit
predictor.find_ques_by_pro(pro_df, pro_tags, expand=True)

132 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%lprun -f predictor.find_ques_by_pro predictor.find_ques_by_pro(pro_df, pro_tags, expand=True)